In [1]:
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)

  |  2023.09.20


In [2]:
import re

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)

print(biz_day)

20230920


In [3]:
import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_stk = {
    'mktId': 'STK',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

print(otp_stk)

q16+eWMBJWHyflE7jp2+pXKc1BJffh2NYO47bYN/0ZQRtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15eUChJWjKN5mCuYuruqGUqmktBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4m8vFLhSKmM794gFu+ThsO31lY4woqehX8j6OlXFDcfHdV4NbYo4+D2Rwcfj24VnU3Zpq3ik/Dyw3FdyOXhJkBI=


In [4]:
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_stk = rq.post(down_url, {'code': otp_stk}, headers=headers)
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')

sector_stk.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,095570,AJ네트웍스,KOSPI,서비스업,4360,-100,-2.24,204145206200
1,006840,AK홀딩스,KOSPI,기타금융,18660,20,0.11,247199488260
2,027410,BGF,KOSPI,기타금융,3630,5,0.14,347451951330
3,282330,BGF리테일,KOSPI,유통업,148600,-600,-0.40,2568388431600
4,138930,BNK금융지주,KOSPI,기타금융,7000,-210,-2.91,2254619066000


In [5]:
gen_otp_ksq = {
    'mktId': 'KSQ',  # 코스닥 입력
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text

down_sector_ksq = rq.post(down_url, {'code': otp_ksq}, headers=headers)
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR')

sector_ksq.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,2575,-5,-0.19,124981853150
1,054620,APS,KOSDAQ,금융,7980,-160,-1.97,162745883580
2,265520,AP시스템,KOSDAQ,반도체,17890,-60,-0.33,273384621690
3,211270,AP위성,KOSDAQ,통신장비,15780,280,1.81,237998757120
4,126600,BGF에코머티리얼즈,KOSDAQ,화학,4860,40,0.83,262601716500


In [6]:
krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)
krx_sector['종목명'] = krx_sector['종목명'].str.strip()
krx_sector['기준일'] = biz_day

krx_sector.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,095570,AJ네트웍스,KOSPI,서비스업,4360,-100,-2.24,204145206200,20230920
1,006840,AK홀딩스,KOSPI,기타금융,18660,20,0.11,247199488260,20230920
2,027410,BGF,KOSPI,기타금융,3630,5,0.14,347451951330,20230920
3,282330,BGF리테일,KOSPI,유통업,148600,-600,-0.40,2568388431600,20230920
4,138930,BNK금융지주,KOSPI,기타금융,7000,-210,-2.91,2254619066000,20230920


In [12]:
krx_sector[krx_sector['종목코드'] == '121890']
# krx_sector[krx_sector['대비'] == 0]

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
1770,121890,에스디시스템,KOSDAQ,정보기기,739,0,0.0,9375042680,20230920


In [ ]:
import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'searchType': '1',
    'mktId': 'ALL',
    'trdDd': biz_day,
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind = rq.post(down_url, {'code': otp}, headers=headers)

krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding='EUC-KR')
krx_ind['종목명'] = krx_ind['종목명'].str.strip()
krx_ind['기준일'] = biz_day

krx_ind.head()

In [ ]:
diff = list(set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명'])))
print(diff)

In [ ]:
kor_ticker = pd.merge(krx_sector,
                      krx_ind,
                      on=krx_sector.columns.intersection(
                          krx_ind.columns).tolist(),
                      how='outer')

kor_ticker.head()

In [ ]:
print(kor_ticker[kor_ticker['종목명'].str.contains('스팩|제[0-9]+호')]['종목명'].values)

In [ ]:
print(kor_ticker[kor_ticker['종목코드'].str[-1:] != '0']['종목명'].values)

In [ ]:
print(kor_ticker[kor_ticker['종목명'].str.endswith('리츠')]['종목명'].values)

In [ ]:
import numpy as np

kor_ticker['종목구분'] = np.where(kor_ticker['종목명'].str.contains('스팩|제[0-9]+호'), '스팩',
                              np.where(kor_ticker['종목코드'].str[-1:] != '0', '우선주',
                                       np.where(kor_ticker['종목명'].str.endswith('리츠'), '리츠',
                                                np.where(kor_ticker['종목명'].isin(diff),  '기타',
                                                '보통주'))))
kor_ticker = kor_ticker.reset_index(drop=True)
kor_ticker.columns = kor_ticker.columns.str.replace(' ', '')
kor_ticker = kor_ticker[['종목코드', '종목명', '시장구분', '종가',
                         '시가총액', '기준일', 'EPS', '선행EPS', 'BPS', '주당배당금', '종목구분']]
kor_ticker = kor_ticker.replace({np.nan: None})
kor_ticker['기준일'] = pd.to_datetime(kor_ticker['기준일'])

kor_ticker.head()

In [ ]:
import sqlite3

conn = sqlite3.connect('./quant_db.sqlite', isolation_level= None)
c = conn.cursor()

In [ ]:
query = """
CREATE TABLE if NOT EXISTS kor_ticker
(
    종목코드 varchar(6) not null,
    종목명 varchar(20),
    시장구분 varchar(6),
    종가 float,
    시가총액 float,
    기준일 date,
    EPS float,
    선행EPS float,
    BPS float,
    주당배당금 float,
    종목구분 varchar(5),
    primary key(종목코드, 기준일)
);
"""
c.execute(query)

In [ ]:
query = f"""
    INSERT OR REPLACE INTO kor_ticker (종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분)
    values (?,?,?,?,?,?,?,?,?,?,?);
"""
kor_ticker['기준일'] = pd.to_datetime(kor_ticker['기준일']).dt.date
args = kor_ticker.values.tolist()
c.executemany(query, args)

# WICS 기준 섹터정보 크롤링

In [ ]:
import json
import requests as rq
import pandas as pd

url = f'''http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={biz_day}&sec_cd=G10'''
data = rq.get(url).json()

type(data)

In [ ]:
print(data.keys())

In [ ]:
data['list'][0]

In [ ]:
data['sector']

In [ ]:
data_pd = pd.json_normalize(data['list'])
data_pd.head()

In [ ]:
import time
import json
import requests as rq
import pandas as pd
from tqdm import tqdm

sector_code = [sec['SEC_CD'] for sec in data['sector']]
# 'G25', 'G35', 'G50', 'G40', 'G10', 'G20', 'G55', 'G30', 'G15', 'G45'

data_sector = []

for i in tqdm(sector_code):
    url = f'''http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={biz_day}&sec_cd={i}'''    
    data = rq.get(url).json()
    data_pd = pd.json_normalize(data['list'])

    data_sector.append(data_pd)

    time.sleep(0.1)

kor_sector = pd.concat(data_sector, axis = 0)
kor_sector = kor_sector[['IDX_CD', 'CMP_CD', 'CMP_KOR', 'SEC_NM_KOR']]
kor_sector['기준일'] = biz_day
# kor_sector['기준일'] = pd.to_datetime(kor_sector['기준일'])

In [ ]:
kor_sector.head()

In [ ]:
kor_sector.size

In [ ]:
query = """
CREATE TABLE if NOT EXISTS kor_sector
(
    IDX_CD varchar(3),
    CMP_CD varchar(6),
    CMP_KOR varchar(20),
    SEC_NM_KOR varchar(10),
    기준일 date,
    primary key(CMP_CD, 기준일)
);
"""
c.execute(query)

In [ ]:
query = f"""
    INSERT OR REPLACE INTO kor_sector (IDX_CD, CMP_CD, CMP_KOR, SEC_NM_KOR, 기준일)
    values (?,?,?,?,?);
"""
kor_sector['기준일'] = biz_day
args = kor_sector.values.tolist()
c.executemany(query, args)

# 수정 주가 크롤링

In [ ]:
query = """
select * from kor_ticker
where 기준일 = (select max(기준일) from kor_ticker) 
	and 종목구분 = '보통주';
"""
ticker_list = pd.read_sql(query, con=conn)

ticker_list.head()

In [ ]:
from dateutil.relativedelta import relativedelta
import requests as rq
from io import BytesIO
from datetime import date

i = 0
ticker = ticker_list['종목코드'][i]
fr = (date.today() + relativedelta(years=-5)).strftime("%Y%m%d")
to = (date.today()).strftime("%Y%m%d")

url = f'''https://fchart.stock.naver.com/siseJson.nhn?symbol={ticker}&requestType=1
&startTime={fr}&endTime={to}&timeframe=day'''

data = rq.get(url).content
data_price = pd.read_csv(BytesIO(data))

data_price.head()

In [ ]:
import re

price = data_price.iloc[:, 0:6]
price.columns = ['날짜', '시가', '고가', '저가', '종가', '거래량']
price = price.dropna()
price['날짜'] = price['날짜'].str.extract('(\d+)')
price['날짜'] = pd.to_datetime(price['날짜'])
price['종목코드'] = ticker

price.head()

In [ ]:
query = """
CREATE TABLE if NOT EXISTS kor_price
(
    날짜 date,
    시가 double,
    고가 double,
    저가 double,
    종가 double,
    거래량 double,
    종목코드 varchar(6),
    primary key(날짜, 종목코드)
);
"""
c.execute(query)

In [ ]:
# 티커리스트 불러오기
ticker_list = pd.read_sql("""
SELECT * FROM kor_ticker
WHERE 기준일 = (select max(기준일) from kor_ticker) AND 종목구분 = '보통주';
""", con=conn)

In [ ]:
from tqdm import tqdm

query = """
    INSERT or IGNORE INTO kor_price (날짜, 시가, 고가, 저가, 종가, 거래량, 종목코드)
    values (?,?,?,?,?,?,?);
"""

error_list = []

# 전종목 주가 다운로드 및 저장
for i in tqdm(range(0, len(ticker_list))):
# 티커 선택
    ticker = ticker_list['종목코드'][i]

    # 시작일과 종료일
    fr = (date.today() + relativedelta(years=-5)).strftime("%Y%m%d")
    to = (date.today()).strftime("%Y%m%d")

    # 오류 발생 시 이를 무시하고 다음 루프로 진행
    try:

        # url 생성
        url = f'''https://fchart.stock.naver.com/siseJson.nhn?symbol={ticker}&requestType=1
        &startTime={fr}&endTime={to}&timeframe=day'''
    
        # 데이터 다운로드
        data = rq.get(url).content
        data_price = pd.read_csv(BytesIO(data))
    
        # 데이터 클렌징
        price = data_price.iloc[:, 0:6]
        price.columns = ['날짜', '시가', '고가', '저가', '종가', '거래량']
        price = price.dropna()
        price['날짜'] = price['날짜'].str.extract('(\d+)')
        price['날짜'] = pd.to_datetime(price['날짜']).dt.date    
        price['종목코드'] = ticker
    
        # 주가 데이터를 DB에 저장
        args = price.values.tolist()
        c.executemany(query, args)

    except:

        # 오류 발생시 error_list에 티커 저장하고 넘어가기
        print(ticker)
        error_list.append(ticker)

    # 타임슬립 적용
    time.sleep(0.1)

# 재무제표 크롤링

In [ ]:
!pip install lxml

In [ ]:
query = """
select * from kor_ticker
where 기준일 = (select max(기준일) from kor_ticker) 
	and 종목구분 = '보통주';
"""
ticker_list = pd.read_sql(query, con=conn)
i = 0
ticker = ticker_list['종목코드'][i]

url = f'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{ticker}'
data = pd.read_html(url, displayed_only=False)

[item.head(3) for item in data]

In [ ]:
print(data[0].columns.tolist(), '\n',
      data[2].columns.tolist(), '\n',
      data[4].columns.tolist()
     )

In [ ]:
data_fs_y = pd.concat(
    [data[0].iloc[:, ~data[0].columns.str.contains('전년동기')], data[2], data[4]])
data_fs_y = data_fs_y.rename(columns={data_fs_y.columns[0]: "계정"})

data_fs_y.head()

In [ ]:
import requests as rq
from bs4 import BeautifulSoup
import re

page_data = rq.get(url)
page_data_html = BeautifulSoup(page_data.content, 'html.parser')

fiscal_data = page_data_html.select('div.corp_group1 > h2')
fiscal_data_text = fiscal_data[1].text
fiscal_data_text = re.findall('[0-9]+', fiscal_data_text)

print(fiscal_data_text)

In [ ]:
data_fs_y = data_fs_y.loc[:, (data_fs_y.columns == '계정') |
                          (data_fs_y.columns.str[-2:].isin(fiscal_data_text))]
data_fs_y.head()

In [ ]:
data_fs_y[data_fs_y.loc[:, ~data_fs_y.columns.isin(['계정'])].isna().all(
    axis=1)].head()

In [ ]:
data_fs_y['계정'].value_counts(ascending=False).head()

In [ ]:
def clean_fs(df, ticker, frequency):

    df = df[~df.loc[:, ~df.columns.isin(['계정'])].isna().all(axis=1)]
    df = df.drop_duplicates(['계정'], keep='first')
    df = pd.melt(df, id_vars='계정', var_name='기준일', value_name='값')
    df = df[~pd.isnull(df['값'])]
    df['계정'] = df['계정'].replace({'계산에 참여한 계정 펼치기': ''}, regex=True)
    df['기준일'] = pd.to_datetime(df['기준일'],
                               format='%Y/%m') + pd.tseries.offsets.MonthEnd()
    df['기준일'] = pd.to_datetime(df['기준일']).dt.date

    df['종목코드'] = ticker
    df['공시구분'] = frequency

    return df

In [ ]:
data_fs_y_clean = clean_fs(data_fs_y, ticker, 'y')

data_fs_y_clean.head()

In [ ]:
# 분기 데이터

data_fs_q = pd.concat(
    [data[1].iloc[:, ~data[1].columns.str.contains('전년동기')], data[3], data[5]])
data_fs_q = data_fs_q.rename(columns={data_fs_q.columns[0]: "계정"})
data_fs_q_clean = clean_fs(data_fs_q, ticker, 'q')

data_fs_q_clean.head()

In [ ]:
data_fs_bind = pd.concat([data_fs_y_clean, data_fs_q_clean])
data_fs_bind.head()

In [ ]:
query = """
CREATE TABLE if NOT EXISTS kor_fs
(
    계정 varchar(30),
    기준일 date,
    값 float,
    종목코드 varchar(6),
    공시구분 varchar(1),
    primary key(계정, 기준일, 종목코드, 공시구분)
);
"""
c.execute(query)

In [ ]:
import pandas as pd
# 티커리스트 불러오기
ticker_list = pd.read_sql("""
select * from kor_ticker
where 기준일 = (select max(기준일) from kor_ticker) 
	and 종목구분 = '보통주';
""", con=conn)

In [ ]:
import requests as rq
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import time

# DB 저장 쿼리
query = """
    INSERT or IGNORE INTO kor_fs (계정, 기준일, 값, 종목코드, 공시구분)
    values (?,?,?,?,?);
"""

# 오류 발생시 저장할 리스트 생성
error_list = []


# 재무제표 클렌징 함수
def clean_fs(df, ticker, frequency):

    df = df[~df.loc[:, ~df.columns.isin(['계정'])].isna().all(axis=1)]
    df = df.drop_duplicates(['계정'], keep='first')
    df = pd.melt(df, id_vars='계정', var_name='기준일', value_name='값')
    df = df[~pd.isnull(df['값'])]
    df['계정'] = df['계정'].replace({'계산에 참여한 계정 펼치기': ''}, regex=True)
    df['기준일'] = pd.to_datetime(df['기준일'],
                               format='%Y/%m') + pd.tseries.offsets.MonthEnd()
    df['기준일'] = pd.to_datetime(df['기준일']).dt.date

    df['종목코드'] = ticker
    df['공시구분'] = frequency

    return df


# for loop
for i in tqdm(range(0, len(ticker_list))):

    # 티커 선택
    ticker = ticker_list['종목코드'][i]

    # 오류 발생 시 이를 무시하고 다음 루프로 진행
    try:

        # url 생성
        url = f'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{ticker}'
    
        # 데이터 받아오기
        data = pd.read_html(url, displayed_only=False)
    
        # 연간 데이터
        data_fs_y = pd.concat([
            data[0].iloc[:, ~data[0].columns.str.contains('전년동기')], data[2],
            data[4]
        ])
        data_fs_y = data_fs_y.rename(columns={data_fs_y.columns[0]: "계정"})
    
        # 결산년 찾기
        page_data = rq.get(url)
        page_data_html = BeautifulSoup(page_data.content, 'html.parser')
    
        fiscal_data = page_data_html.select('div.corp_group1 > h2')
        fiscal_data_text = fiscal_data[1].text
        fiscal_data_text = re.findall('[0-9]+', fiscal_data_text)
    
        # 결산년에 해당하는 계정만 남기기
        data_fs_y = data_fs_y.loc[:, (data_fs_y.columns == '계정') | (
            data_fs_y.columns.str[-2:].isin(fiscal_data_text))]
    
        # 클렌징
        data_fs_y_clean = clean_fs(data_fs_y, ticker, 'y')
    
        # 분기 데이터
        data_fs_q = pd.concat([
            data[1].iloc[:, ~data[1].columns.str.contains('전년동기')], data[3],
            data[5]
        ])
        data_fs_q = data_fs_q.rename(columns={data_fs_q.columns[0]: "계정"})
    
        data_fs_q_clean = clean_fs(data_fs_q, ticker, 'q')
    
        # 두개 합치기
        data_fs_bind = pd.concat([data_fs_y_clean, data_fs_q_clean])
        data_fs_bind = data_fs_bind.dropna()
        # print(data_fs_bind.head())
    
        # 재무제표 데이터를 DB에 저장
        args = data_fs_bind.values.tolist()
        # print(args[0])
        c.executemany(query, args)

    except:

        # 오류 발생시 해당 종목명을 저장하고 다음 루프로 이동
        print(ticker)
        error_list.append(ticker)

    # 타임슬립 적용
    # time.sleep(0.1)

# 가치지표 계산

In [1]:
import sqlite3

conn = sqlite3.connect('./quant_db.sqlite', isolation_level= None)
c = conn.cursor()

In [30]:
import pandas as pd

# 티커 리스트
ticker_list = pd.read_sql("""
select * from kor_ticker
where 기준일 = (select max(기준일) from kor_ticker) 
	and 종목구분 = '보통주';
""", con=conn)

# 삼성전자 분기 재무제표
sample_fs = pd.read_sql("""
select * from kor_fs
where 공시구분 = 'q'
and 종목코드 = '005930'
and 계정 in ('당기순이익', '자본', '영업활동으로인한현금흐름', '매출액');
""", con=conn)

In [31]:
sample_fs = sample_fs.sort_values(['종목코드', '계정', '기준일'])

sample_fs.head()

,계정,기준일,값,종목코드,공시구분
0,당기순이익,2022-09-30,93892.0,005930,q
1,당기순이익,2022-12-31,238414.0,005930,q
2,당기순이익,2023-03-31,15746.0,005930,q
3,당기순이익,2023-06-30,17236.0,005930,q
4,매출액,2022-09-30,767817.0,005930,q


In [32]:
sample_fs['ttm'] = sample_fs.groupby(
    ['종목코드', '계정'], as_index=False)['값'].rolling(window=4,
                                                 min_periods=4).sum()['값']

sample_fs

,계정,기준일,값,종목코드,공시구분,ttm
0,당기순이익,2022-09-30,93892.0,005930,q,NaN
1,당기순이익,2022-12-31,238414.0,005930,q,NaN
2,당기순이익,2023-03-31,15746.0,005930,q,NaN
3,당기순이익,2023-06-30,17236.0,005930,q,365288.0
4,매출액,2022-09-30,767817.0,005930,q,NaN
5,매출액,2022-12-31,704646.0,005930,q,NaN
6,매출액,2023-03-31,637454.0,005930,q,NaN
7,매출액,2023-06-30,600055.0,005930,q,2709972.0
8,영업활동으로인한현금흐름,2022-09-30,189793.0,005930,q,NaN
9,영업활동으로인한현금흐름,2022-12-31,186129.0,005930,q,NaN


In [33]:
import numpy as np

sample_fs['ttm'] = np.where(sample_fs['계정'] == '자본',
                            sample_fs['ttm'] / 4, sample_fs['ttm'])
sample_fs = sample_fs.groupby(['계정', '종목코드']).tail(1)

sample_fs.head()

,계정,기준일,값,종목코드,공시구분,ttm
3,당기순이익,2023-06-30,17236.0,005930,q,365288.00
7,매출액,2023-06-30,600055.0,005930,q,2709972.00
11,영업활동으로인한현금흐름,2023-06-30,81699.0,005930,q,520539.00
15,자본,2023-06-30,3589756.0,005930,q,3546078.75


In [34]:
sample_fs_merge = sample_fs[['계정', '종목코드', 'ttm']].merge(
    ticker_list[['종목코드', '시가총액', '기준일']], on='종목코드')
sample_fs_merge['시가총액'] = sample_fs_merge['시가총액']/100000000

sample_fs_merge.head()

,계정,종목코드,ttm,시가총액,기준일
0,당기순이익,005930,365288.00,4.166908e+06,2023-09-19
1,매출액,005930,2709972.00,4.166908e+06,2023-09-19
2,영업활동으로인한현금흐름,005930,520539.00,4.166908e+06,2023-09-19
3,자본,005930,3546078.75,4.166908e+06,2023-09-19


In [36]:
sample_fs_merge['value'] = sample_fs_merge['시가총액'] / sample_fs_merge['ttm']
sample_fs_merge['지표'] = \
    np.where(sample_fs_merge['계정'] == '매출액', 'PSR', \
        np.where(sample_fs_merge['계정'] == '영업활동으로인한현금흐름', 'PCR', \
            np.where(sample_fs_merge['계정'] == '자본', 'PBR', \
                 np.where(sample_fs_merge['계정'] == '당기순이익', 'PER', None) \
            ) \
        ) \
    )
sample_fs_merge

,계정,종목코드,ttm,시가총액,기준일,value,지표
0,당기순이익,005930,365288.00,4.166908e+06,2023-09-19,11.407186,PER
1,매출액,005930,2709972.00,4.166908e+06,2023-09-19,1.537620,PSR
2,영업활동으로인한현금흐름,005930,520539.00,4.166908e+06,2023-09-19,8.004988,PCR
3,자본,005930,3546078.75,4.166908e+06,2023-09-19,1.175075,PBR


In [37]:
ticker_list_sample = ticker_list[ticker_list['종목코드'] == '005930'].copy()
ticker_list_sample['DY'] = ticker_list_sample['주당배당금'] / ticker_list_sample['종가']

ticker_list_sample.head()

,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분,DY
366,005930,삼성전자,KOSPI,69800.0,4.166908e+14,2023-09-19,8057.0,3887.0,57822.0,1444.0,보통주,0.020688
